In [0]:
src_path="dbfs:/databricks-datasets/weather/high_temps"

In [0]:
dbutils.fs.ls("dbfs:/databricks-datasets/nyctaxi/tripdata/green/")

In [0]:
from pyspark.sql.functions import spark_partition_id, col

df_taxi_gree=spark.read.format("csv").option("header",True).load("dbfs:/databricks-datasets/nyctaxi/tripdata/green/*")


df_taxi_gree.display()

In [0]:
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField
from pyspark.sql.functions import spark_partition_id, col
data = [("1", i) for i in range (100000)]+\
[("2", i) for i in range (90000)]+\
[("3", i) for i in range (1000)]

schema=StructType([StructField("cid", StringType(), True),
                  StructField("orderid", IntegerType(), True)])
df_skew=spark.createDataFrame(data,schema)
df_skew=df_skew.repartition(5, col("orderid"))
df_skew.withColumn("partitionID", spark_partition_id()).groupBy(col("partitionID")).count().display()

In [0]:
df_skew.display()

In [0]:
from pyspark.sql.functions import concat, lit, rand
df_taxi_gree_salt= df_taxi_gree.withColumn("salt", concat(col("VendorID"), lit("_"), (rand()*5).cast("int")))
df_taxi_gree_salt.select(col("VendorID"), col("salt")).display()

In [0]:
from pyspark.sql.functions import explode, array
df_skew_salt=df_skew.withColumn("skewsalt",  explode(array([lit(i) for i in range(5)]))).withColumn("skew_salt", concat(col("cid"), lit("_"), col("skewsalt"))).drop("skewsalt")
df_skew_salt.display()

In [0]:
df_join_skew= df_taxi_gree_salt.join(df_skew_salt, df_taxi_gree_salt.salt==df_skew_salt.skew_salt, "inner")
df_join_skew.display()